#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com), modified by [Seabass](https://github.com/pickles976)

*Last updated: May 1sr, 2022 (aitextgen v0.5.2)*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [ ]:
# !pip install -q aitextgen==0.4.0

!pip install -q transformers>=4.0.0
!pip install -q fire>=0.3.0
!pip install -q pytorch-lightning>=1.0.8
!pip install -q tokenizers>=1.0.0
!pip install -q torch>=1.6.0
!pip install -q aitextgen --no-deps

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

     |████████████████████████████████| 572 kB 4.2 MB/s 


04/30/2022 16:33:38 — INFO — numexpr.utils — NumExpr defaulting to 4 threads.


## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [ ]:
!nvidia-smi

Sat Apr 30 16:33:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/w3wvHhR.png)

Upload **any smaller text file** (for example, [a text file of Shakespeare plays](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)) and update the file name in the cell below, then run the cell.

In [ ]:
# file_name = "/content/lakh_dataset.txt"
file_name = "/content/abc_8192.txt"

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

In [ ]:
copy_file_from_gdrive(file_name)

# Train From Scratch

In [ ]:
from aitextgen.tokenizers import train_tokenizer
train_tokenizer(file_name, vocab_size=8192,serialize=True)

In [ ]:
# If you want to load a partially-trained model
ai = aitextgen(model_folder="/content/drive/MyDrive/NEO_NES_1000",
                tokenizer_file="aitextgen.tokenizer.json")

In [ ]:
from aitextgen.TokenDataset import TokenDataset
data = TokenDataset(file_name, block_size=2048)

**ONLY USE ON FIRST RUN**
building a new model and tokenizer

In [ ]:
from aitextgen.utils import build_gpt2_config
tokenizer_file = "aitextgen.tokenizer.json"
config = build_gpt2_config(vocab_size=8192, max_length=2048, dropout=0.0, n_embd=768, n_layer=12, n_head=8) # 92M param
ai = aitextgen(tokenizer_file=tokenizer_file, config=config)

In [ ]:
ai.train(file_name,
         line_by_line=False,
         from_cache=False,
         num_steps=1e5,
         generate_every=5000,
         save_every=5000,
         save_gdrive=True,
         learning_rate=1e-5,
         fp16=False,
         batch_size=1
         )

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

Load previously generated model

In [ ]:
from aitextgen.tokenizers import train_tokenizer
file_name = "/content/abc_8192.txt"
from_folder = "/content/drive/MyDrive/NEO_LAKH_1092"

train_tokenizer(file_name, vocab_size=8192,serialize=True)
ai = aitextgen(model_folder=from_folder,
                tokenizer_file="aitextgen.tokenizer.json")

04/30/2022 16:36:48 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/MyDrive/NEO_LAKH_1092.
04/30/2022 16:36:53 — INFO — aitextgen — GPT2 loaded with 92M parameters.
04/30/2022 16:36:53 — INFO — aitextgen — Using a custom tokenizer.


In [ ]:
file_name = "/content/abc_2048.txt"
ai.train(file_name,
         line_by_line=False,
         from_cache=False,
         num_steps=0.5e5,
         generate_every=5000,
         save_every=5000,
         save_gdrive=True,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

04/30/2022 16:39:53 — INFO — aitextgen — Loading text from /content/abc_2048.txt with generation length of 2048.


  0%|          | 0/91959 [00:00<?, ?it/s]

04/30/2022 16:39:53 — INFO — aitextgen.TokenDataset — Encoding 91,959 sets of tokens from /content/abc_2048.txt.
04/30/2022 16:39:56 — WARNING — aitextgen — pytorch_model.bin already exists in /trained_model and will be overwritten!
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to 

  0%|          | 0/50000.0 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2265: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  "`trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7."


5,000 steps reached: saving model to /trained_model
5,000 steps reached: generating sample texts.

X:1
T:Music21 Fragment
C:Music21
%%score 1 2 2 3 4
L:1/16
Q:1/4=120
M:4/4
I:linebreak $
K:C
V:1 treble nm="Violin" snm="Vln"
V:2 treble nm="Viola" snm="Vla"
V:3 treble nm="Celogue 2" snm="Samp"
V:4 bass nm="Tuba" snm="Tba"
V:1
 c z eg z g f z e z g z g z f z e z | c z dg z g f z e z g z g z f z e z | %2
 c z eg z g f z e z g z g z f z e z | c z dg z g f z e z g z g z f z e z | %4
 c z eg z g f z e z g z g z f z e z | c z dg z g f z e z g z g z f z e z | %6
 c z eg z g f z e z g z g z f z e z | c z dg z g f z
10,000 steps reached: saving model to /trained_model
10,000 steps reached: generating sample texts.
:linebreak $
K:F
V:1 treble nm="Harpsichord" snm="Hpschd"
V:2 bass 
V:1
 [DG]2[DG]2 [DG]2[DG]2 [DG]2[DG]2 [DG]2[CF]2 | [CG]2[CG]2 [CF]2[CG]2 [CF]2[CG]2 [CG]2[CG]2 | %2
 [CF]2[CG]2 [CF]2[CG]2 [CF]2[CG]2 [CF]2[CG]2 | [CF]2[CG]2 [CF]2[CG]2 [CF]2[CG]2 [CF]2[CG]2 | %4
 [CF]2[CG]2 [CF]2[CG]2 

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
04/30/2022 19:31:51 — INFO — aitextgen — Saving trained model pytorch_model.bin to /trained_model


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


## Load a Trained Model

If you already had a trained model from this notebook, running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
from aitextgen.tokenizers import train_tokenizer
file_name = "/content/abc_8192.txt"

train_tokenizer(file_name, vocab_size=8192,serialize=True)

In [ ]:
from_folder = "/content/drive/MyDrive/NEO_LAKH_1092"

for file in ["pytorch_model.bin", "config.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

The next cell will allow you to load the retrained model + metadata necessary to generate text.

In [ ]:
ai = aitextgen(model_folder=".",to_gpu=True)

04/30/2022 19:32:36 — INFO — aitextgen — Loading model from provided weights and config in /..
04/30/2022 19:32:37 — INFO — aitextgen — GPT2 loaded with 92M parameters.
04/30/2022 19:32:37 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model**, you'll get much faster training performance if you reload the model; the next cell will reload the model you just trained from the `trained_model` folder.

In [ ]:
tokenizer = "/content/aitextgen.tokenizer.json"
ai = aitextgen(model_folder=".",tokenizer_file=tokenizer,to_gpu=True)

04/30/2022 19:32:42 — INFO — aitextgen — Loading model from provided weights and config in /..
04/30/2022 19:32:43 — INFO — aitextgen — GPT2 loaded with 92M parameters.
04/30/2022 19:32:43 — INFO — aitextgen — Using a custom tokenizer.


`generate()` without any parameters generates a single text from the loaded model to the console.

In [ ]:
prompt="""X:1
T:Music21 Fragment
C:Music21
%%score 1 2 3 4
L:1/8
Q:1/4=180
M:4/4
I:linebreak $
K:none
V:1 treble nm="Brass" snm="Brs"
V:2 treble nm="Brass" snm="Brs"
V:3 bass nm="Fretless Bass" snm="Gtr"
V:4 bass nm="Percussion" snm="Perc"
"""

ai.generate(prompt=prompt,max_length=2048,temperature=0.9)

X:1
T:Music21 Fragment
C:Music21
%%score 1 2 3 4
L:1/8
Q:1/4=180
M:4/4
I:linebreak $
K:none
V:1 treble nm="Brass" snm="Brs"
V:2 treble nm="Brass" snm="Brs"
V:3 bass nm="Fretless Bass" snm="Gtr"
V:4 bass nm="Percussion" snm="Perc"
V:1
 _ee z b ^c' z _ee z b ^c' z _e'f' | _e'_e'^c'=c' _b z =b z _e'f' _e' z ^g'b' | %2
 =c''2 _b' z =g z ^c' z =c' z ^g z c' z | _e' z b ^c' z =c' z _b^g g z ^f z =f z | %4
 _ee z b ^c' z _ee z b ^c' z _e'f' | _e'_e'^c'=c' _b z =b z _e'f' _e' z ^g'b' | %6
 =c''2 _b' z =g' z ^c' z =c' z ^g z c' z | _e' z b ^c' z =c' z _b^g g z ^f z =f z | %8
V:2
 A,2 A,2 B, B,, | C,2 C2 ^C D | _E2 E2 _E =E | ^F2 F2 ^F =F | ^F2 F2 ^F =F | ^F2 F2 F ^F | %6
 ^G2 G2 ^F ^G | A2 A2 ^F ^G | A4 z4 | %10
V:3
 ^G,, z G,, z G,, z G,, z G,, z G,, z G,, z | ^G,, z G,, z G,, z G,, z G,, z7 | %2
 A,, z A,, z A,, z A,, z A,, z A,, z A,, z | A,, z A,, z A,, z A,, z A,, z7 | %4
 A,, z A,, z A,, z A,, z A,, z A,, z A,, z | A,, z A,, z A,, z A,, z A,, z7 | %6
 A,2A,2 A,2 B, B,,4 z4 | E,, z E,, z E

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_files = 5

for _ in range(num_files):
  ai.generate_to_file(n=1000,
                     batch_size=50,
                     prompt=prompt,
                     max_length=256,
                     temperature=1.0,
                     top_p=0.9)

# LICENSE

MIT License

Copyright (c) 2020-2021 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.